# Audio Classification - Environmental Sounds - CNN-DNN-Librosa

We are going to use a subset of the data from ESC-50 dataset from https://dagshub.com/kinkusuma/esc50-dataset. The ESC-50 dataset is a labeled collection of 2000 environmental audio recordings suitable for benchmarking methods of environmental sound classification.
We will develop and train a model to classify 8 differnet environment sounds from the above dataset that has 50+ environment sound audio files for classification.

# Common Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.image import resize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
import librosa
from sklearn.model_selection import train_test_split
import os
import numpy as np

# Defining labels for classification

In [2]:
audio_data_path=r'C:\\Users\\Yoffi Nelluri\\Raju AI Kadi\\voice\\kaggle\\archive\\KAGGLE\AUDIO'
inference_categories=os.listdir(audio_data_path)
category_count=len(inference_categories)

In [3]:
print (inference_categories)

['FAKE', 'REAL']


# Data Processing

In [4]:
# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(200, 200)):
    data = []
    labels = []
    
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)

In [5]:
# Split data into training and testing sets
data, labels = load_and_preprocess_data(audio_data_path, inference_categories)
labels = to_categorical(labels, num_classes=len(inference_categories))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [6]:
X_train[0].shape

(200, 200, 1)

# Defining callbacks

In [7]:
class myCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training when the loss falls below 0.1

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''
    # Check the loss
    if(logs.get('loss') < 0.1):
      # Stop if threshold is met
      print("\nLoss is lower than 0.1 so cancelling training!")
      self.model.stop_training = True
                
    
# Instantiate class
callbacks = myCallback()

# Creating and compiling model

In [8]:
def create_model():
    model=Sequential([
        Conv2D(64,(3,3),activation='relu',input_shape=X_train[0].shape),
        MaxPooling2D(2,2),
        Conv2D(128,(3,3),activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(256,(3,3),activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512,activation='relu'),
        Dropout(0.3),
        Dense(256,activation='relu'),
        Dropout(0.3),
        Dense(category_count,activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
    return model

# Training the model

In [9]:
model=create_model()
model.fit(X_train,y_train,epochs=100,verbose=1,batch_size=25,validation_data=(X_test,y_test),callbacks=[callbacks])

c:\Users\Yoffi Nelluri\Raju AI Kadi\voice\GOF_hackathon_ImpersV3\hack7\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5487 - loss: 6.1867 - val_accuracy: 0.7692 - val_loss: 24.4710
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.8960 - loss: 10.7338 - val_accuracy: 0.2308 - val_loss: 6.8463
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.6187 - loss: 6.6832 - val_accuracy: 0.7692 - val_loss: 1.6546
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8860 - loss: 1.6120 - val_accuracy: 0.7692 - val_loss: 0.9696
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8960 - loss: 0.4973 - val_accuracy: 0.7692 - val_loss: 0.5853
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9158 - loss: 0.2689 - val_accuracy: 0.7692 - val_loss: 0.4938
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9556 - loss: 0.1182 - val_accuracy: 0.7692 - val_loss: 0.5078
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9654 - loss: 0.2363 - val_accuracy: 0.7692 - val_loss: 0.6026
Epoch

# Save the model

In [10]:
#Validate the model on test dataset (X_test,y_test) 
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

# Save the model
model.save('FinalModel.h5')

0.7692307829856873


# Evaluate with a random .wav file data 

In [12]:
# Load the saved model
#model = load_model('audio_classification_model.h5')
##model = load_model('audio_classification_model.h5')
# Define the target shape for input spectrograms
target_shape = (200, 200)
inference_categories = ['FAKE','REAL']
                      
# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))

    # Make predictions
    model2 = load_model("FinalModel.h5")
    predictions = model2.predict(mel_spectrogram)

    # Get the class probabilities
    class_probabilities = predictions[0]

    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)

    return class_probabilities, predicted_class_index

# Test an audio file - FAKE
#test_audio_file = 'C:\\Users\Yoffi Nelluri\\Raju AI Kadi\\voice\\challenge-assets-main\\challenge-assets-main\\identity-personification\\synthetic-audio-train\\audio_file (2).wav'
#test_audio_file = 'C:\\Users\\Yoffi Nelluri\\Raju AI Kadi\\voice\\challenge-assets-main\\challenge-assets-main\\identity-personification\\synthetic-audio-train\\1.3_Audio_HM.wav'
#test_audio_file = 'C:\\Users\\Yoffi Nelluri\\Raju AI Kadi\\voice\\kaggle\\archive\\DEMONSTRATION\\DEMONSTRATION\\linus-original-DEMO.mp3'
test_audio_file = 'C:\\Users\\Yoffi Nelluri\\Raju AI Kadi\\voice\\kaggle\\archive\\DEMONSTRATION\\DEMONSTRATION\\linus-to-musk-DEMO.mp3'
# Test an audio file - FAKE
#test_audio_file = 'C:\\Users\Yoffi Nelluri\\Raju AI Kadi\\voice\\challenge-assets-main\\challenge-assets-main\\identity-personification\\synthetic-audio-train\\audio_file (2).wav'
#test_audio_file = 'C:\\Users\Yoffi Nelluri\\Raju AI Kadi\\voice\\challenge-assets-main\\challenge-assets-main\\identity-personification\\human_voice_train_data2\\human_voice_Script3.mp3'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(inference_categories):
    probability = class_probabilities[i]
    #print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = inference_categories[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
if accuracy<0.75:
    predicted_class ='REAL'
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 583ms/step
The audio is classified as: FAKE
Accuracy: 0.9999


In [9]:
class_probabilities = predictions_out[0] 
predicted_class_index = np.argmax(class_probabilities)
print (predicted_class_index)
#result = 90                    
st.info(f"Result probability: {predicted_class_index * 100:.2f}")
st.success(f"The uploaded audio is {predicted_class_index * 100:.2f}% likely to be {class_probabilities}")

AttributeError: module 'datetime' has no attribute 'now'